## Regresja liniowa i wielomianowa w wycenie dyni - Lekcja 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografika autorstwa Dasani Madipalli</figcaption>


<!--![Infografika autorstwa Dasani Madipalli](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.pl.png){width="800"}-->

#### Wprowadzenie

Do tej pory zapoznałeś się z pojęciem regresji, korzystając z przykładowych danych z zestawu dotyczącego wyceny dyni, który będziemy używać w tej lekcji. Zobrazowałeś je również za pomocą `ggplot2`. 💪

Teraz jesteś gotowy, aby zagłębić się w temat regresji w kontekście uczenia maszynowego. W tej lekcji dowiesz się więcej o dwóch typach regresji: *podstawowej regresji liniowej* oraz *regresji wielomianowej*, a także o matematyce stojącej za tymi technikami.

> W całym tym kursie zakładamy minimalną znajomość matematyki i staramy się uczynić ją przystępną dla studentów z innych dziedzin, dlatego zwracaj uwagę na notatki, 🧮 wskazówki, diagramy i inne narzędzia edukacyjne, które pomogą w zrozumieniu materiału.

#### Przygotowanie

Przypominając, wczytujesz te dane, aby móc zadawać im pytania.

-   Kiedy jest najlepszy czas na zakup dyni?

-   Jakiej ceny mogę się spodziewać za skrzynkę miniaturowych dyń?

-   Czy powinienem kupować je w koszach o pojemności pół buszla czy w pudełkach o pojemności 1 1/9 buszla? Zobaczmy, co jeszcze możemy odkryć w tych danych.

W poprzedniej lekcji stworzyłeś `tibble` (nowoczesne podejście do ramek danych) i wypełniłeś go częścią oryginalnego zestawu danych, standaryzując ceny według buszla. Jednakże, w ten sposób udało Ci się zebrać jedynie około 400 punktów danych i tylko dla jesiennych miesięcy. Może uda nam się uzyskać więcej szczegółów na temat charakteru danych, jeśli je bardziej oczyścimy? Zobaczymy... 🕵️‍♀️

Do tego zadania będziemy potrzebować następujących pakietów:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) to [zbiór pakietów R](https://www.tidyverse.org/packages) zaprojektowany, aby uczynić analizę danych szybszą, łatwiejszą i bardziej przyjemną!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) to [framework](https://www.tidymodels.org/packages/) składający się z pakietów do modelowania i uczenia maszynowego.

-   `janitor`: [pakiet janitor](https://github.com/sfirke/janitor) oferuje proste narzędzia do badania i czyszczenia "brudnych" danych.

-   `corrplot`: [pakiet corrplot](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) zapewnia wizualne narzędzie eksploracyjne dla macierzy korelacji, wspierające automatyczne porządkowanie zmiennych w celu wykrycia ukrytych wzorców między zmiennymi.

Możesz je zainstalować za pomocą:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Poniższy skrypt sprawdza, czy masz zainstalowane wymagane pakiety do ukończenia tego modułu i instaluje je w razie ich braku.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Załadujemy później te świetne pakiety i udostępnimy je w naszej bieżącej sesji R. (To tylko ilustracja, `pacman::p_load()` już to dla Ciebie zrobił)

## 1. Linia regresji liniowej

Jak nauczyłeś się w Lekcji 1, celem ćwiczenia regresji liniowej jest możliwość narysowania *linii* *najlepszego dopasowania*, aby:

-   **Pokazać zależności między zmiennymi**. Zobrazować relacje między zmiennymi.

-   **Dokonywać prognoz**. Dokonywać dokładnych prognoz, gdzie nowy punkt danych znajdzie się w odniesieniu do tej linii.

Aby narysować tego typu linię, używamy techniki statystycznej zwanej **Regresją Metodą Najmniejszych Kwadratów**. Termin `najmniejsze kwadraty` oznacza, że wszystkie punkty danych otaczające linię regresji są podnoszone do kwadratu, a następnie sumowane. Idealnie, ta końcowa suma powinna być jak najmniejsza, ponieważ chcemy mieć jak najmniej błędów, czyli `najmniejsze kwadraty`. W związku z tym linia najlepszego dopasowania to linia, która daje nam najniższą wartość sumy kwadratów błędów - stąd nazwa *regresja metodą najmniejszych kwadratów*.

Robimy to, ponieważ chcemy modelować linię, która ma najmniejszą skumulowaną odległość od wszystkich naszych punktów danych. Podnosimy wartości do kwadratu przed ich dodaniem, ponieważ interesuje nas ich wielkość, a nie kierunek.

> **🧮 Pokaż mi matematykę**
>
> Ta linia, zwana *linią najlepszego dopasowania*, może być wyrażona za pomocą [równania](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` to '`zmienna objaśniająca` lub `predyktor`'. `Y` to '`zmienna zależna` lub `wynik`'. Nachylenie linii to `b`, a `a` to punkt przecięcia z osią Y, który odnosi się do wartości `Y`, gdy `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "nachylenie = $y/x$")
    Infografika autorstwa Jen Looper
>
> Najpierw oblicz nachylenie `b`.
>
> Innymi słowy, odnosząc się do pierwotnego pytania dotyczącego danych o dyniach: "przewidzieć cenę dyni za buszel w zależności od miesiąca", `X` odnosiłoby się do ceny, a `Y` do miesiąca sprzedaży.
>
> ![](../../../../../../translated_images/calculation.989aa7822020d9d0ba9fc781f1ab5192f3421be86ebb88026528aef33c37b0d8.pl.png)
    Infografika autorstwa Jen Looper
> 
> Oblicz wartość Y. Jeśli płacisz około 4$, to musi być kwiecień!
>
> Matematyka, która oblicza linię, musi uwzględniać nachylenie linii, które również zależy od punktu przecięcia, czyli miejsca, gdzie `Y` znajduje się, gdy `X = 0`.
>
> Możesz zobaczyć metodę obliczania tych wartości na stronie [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html). Odwiedź również [ten kalkulator regresji metodą najmniejszych kwadratów](https://www.mathsisfun.com/data/least-squares-calculator.html), aby zobaczyć, jak wartości liczb wpływają na linię.

Nie takie straszne, prawda? 🤓

#### Korelacja

Jeszcze jedno pojęcie do zrozumienia to **Współczynnik Korelacji** między danymi zmiennymi X i Y. Korzystając z wykresu punktowego, możesz szybko zobrazować ten współczynnik. Wykres z punktami danych ułożonymi w schludną linię ma wysoką korelację, ale wykres z punktami danych rozrzuconymi wszędzie między X i Y ma niską korelację.

Dobry model regresji liniowej to taki, który ma wysoki (bliższy 1 niż 0) Współczynnik Korelacji, korzystając z metody regresji metodą najmniejszych kwadratów z linią regresji.


## **2. Taniec z danymi: tworzenie ramki danych do modelowania**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Grafika autorstwa @allison_horst</figcaption>


<!--![Grafika autorstwa \@allison_horst](../../../../../../translated_images/janitor.e4a77dd3d3e6a32e25327090b8a9c00dc7cf459c44fa9f184c5ecb0d48ce3794.pl.jpg){width="700"}-->


Załaduj wymagane biblioteki i zestaw danych. Przekształć dane w ramkę danych zawierającą podzbiór danych:

-   Uwzględnij tylko dynie wyceniane na podstawie buszla

-   Przekształć datę na miesiąc

-   Oblicz cenę jako średnią cen wysokich i niskich

-   Przekształć cenę, aby odzwierciedlała wycenę według ilości buszli

> Omówiliśmy te kroki w [poprzedniej lekcji](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

W duchu czystej przygody, przyjrzyjmy się [`pakietowi janitor`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor), który oferuje proste funkcje do badania i czyszczenia brudnych danych. Na przykład, spójrzmy na nazwy kolumn w naszych danych:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Możemy zrobić to lepiej. Zmieńmy nazwy tych kolumn na `friendR`, konwertując je na konwencję [snake_case](https://en.wikipedia.org/wiki/Snake_case) za pomocą `janitor::clean_names`. Aby dowiedzieć się więcej o tej funkcji: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Dużo porządku 🧹! Teraz taniec z danymi przy użyciu `dplyr`, tak jak w poprzedniej lekcji! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Dobra robota!👌 Masz teraz czysty, uporządkowany zestaw danych, na którym możesz oprzeć swój nowy model regresji!

Co powiesz na wykres punktowy?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Wykres punktowy przypomina nam, że posiadamy dane miesięczne tylko od sierpnia do grudnia. Prawdopodobnie potrzebujemy więcej danych, aby móc wyciągać wnioski w sposób liniowy.

Spójrzmy ponownie na nasze dane modelowe:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Co jeśli chcielibyśmy przewidzieć `cenę` dyni na podstawie kolumn `miasto` lub `opakowanie`, które są typu znakowego? A nawet bardziej podstawowo, jak moglibyśmy znaleźć korelację (która wymaga, aby oba jej wejścia były numeryczne) między, na przykład, `opakowaniem` a `ceną`? 🤷🤷

Modele uczenia maszynowego najlepiej działają z cechami numerycznymi, a nie wartościami tekstowymi, więc zazwyczaj trzeba przekształcić cechy kategoryczne w reprezentacje numeryczne.

Oznacza to, że musimy znaleźć sposób na przekształcenie naszych predyktorów, aby były bardziej użyteczne dla modelu, proces ten nazywany jest `inżynierią cech`.


## 3. Przygotowanie danych do modelowania za pomocą recipes 👩‍🍳👨‍🍳

Działania, które przekształcają wartości predyktorów, aby model mógł je skuteczniej wykorzystać, nazywane są `inżynierią cech`.

Różne modele mają różne wymagania dotyczące przetwarzania danych. Na przykład metoda najmniejszych kwadratów wymaga `kodowania zmiennych kategorycznych`, takich jak miesiąc, odmiana czy city_name. Polega to na `przekształceniu` kolumny z `wartościami kategorycznymi` w jedną lub więcej `kolumn numerycznych`, które zastępują oryginalną kolumnę.

Na przykład, jeśli Twoje dane zawierają następującą cechę kategoryczną:

|  miasto   |
|:---------:|
| Denver    |
| Nairobi   |
|  Tokio    |

Możesz zastosować *kodowanie porządkowe*, aby zastąpić każdą kategorię unikalną wartością całkowitą, tak jak tutaj:

| miasto |
|:------:|
|   0    |
|   1    |
|   2    |

I właśnie to zrobimy z naszymi danymi!

W tej sekcji przyjrzymy się kolejnemu niesamowitemu pakietowi Tidymodels: [recipes](https://tidymodels.github.io/recipes/) - który został zaprojektowany, aby pomóc w przetwarzaniu danych **przed** trenowaniem modelu. W swojej istocie, recipe to obiekt, który definiuje, jakie kroki należy zastosować do zestawu danych, aby przygotować go do modelowania.

Teraz stwórzmy recipe, która przygotuje nasze dane do modelowania, zastępując unikalną wartością całkowitą wszystkie obserwacje w kolumnach predyktorów:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Super! 👏 Właśnie stworzyliśmy nasz pierwszy przepis, który określa wynik (cena) oraz odpowiadające mu predyktory, a także to, że wszystkie kolumny predyktorów powinny być zakodowane jako zestaw liczb całkowitych 🙌! Szybko to rozłóżmy na części:

-   Wywołanie `recipe()` z formułą informuje przepis o *rolach* zmiennych, używając danych `new_pumpkins` jako odniesienia. Na przykład kolumna `price` została przypisana do roli `outcome`, podczas gdy pozostałe kolumny zostały przypisane do roli `predictor`.

-   `step_integer(all_predictors(), zero_based = TRUE)` określa, że wszystkie predyktory powinny zostać przekształcone w zestaw liczb całkowitych, zaczynając od numeracji 0.

Jesteśmy pewni, że możesz mieć takie myśli jak: "To jest takie świetne!! Ale co jeśli chciałbym upewnić się, że przepisy robią dokładnie to, czego od nich oczekuję? 🤔"

To świetna myśl! Widzisz, gdy Twój przepis jest zdefiniowany, możesz oszacować parametry potrzebne do faktycznego przetworzenia danych, a następnie wyodrębnić przetworzone dane. Zazwyczaj nie musisz tego robić, gdy używasz Tidymodels (zaraz zobaczymy normalną konwencję -\> `workflows`), ale może się to przydać, gdy chcesz przeprowadzić pewnego rodzaju kontrolę poprawności, aby upewnić się, że przepisy działają zgodnie z oczekiwaniami.

Do tego potrzebujesz dwóch dodatkowych czasowników: `prep()` i `bake()`, a jak zawsze, nasi mali przyjaciele z R od [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) pomogą Ci lepiej to zrozumieć!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Ilustracja autorstwa @allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): szacuje wymagane parametry na podstawie zestawu treningowego, które później mogą być zastosowane do innych zestawów danych. Na przykład, dla danej kolumny predyktora, które obserwacje zostaną przypisane jako liczba całkowita 0, 1, 2 itd.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): bierze przygotowany przepis i stosuje operacje do dowolnego zestawu danych.

Mając to na uwadze, przygotujmy i zastosujmy nasze przepisy, aby naprawdę potwierdzić, że w tle kolumny predyktorów zostaną najpierw zakodowane, zanim model zostanie dopasowany.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Hurra! 🥳 Przetworzone dane `baked_pumpkins` mają wszystkie predyktory zakodowane, co potwierdza, że kroki przetwarzania wstępnego zdefiniowane w naszym przepisie działają zgodnie z oczekiwaniami. Może to być trudniejsze do odczytania dla Ciebie, ale za to znacznie bardziej zrozumiałe dla Tidymodels! Poświęć chwilę, aby sprawdzić, która obserwacja została przypisana do odpowiedniej liczby całkowitej.

Warto również wspomnieć, że `baked_pumpkins` to ramka danych, na której możemy wykonywać obliczenia.

Na przykład spróbujmy znaleźć dobrą korelację między dwoma punktami w Twoich danych, aby potencjalnie zbudować dobry model predykcyjny. Użyjemy do tego funkcji `cor()`. Wpisz `?cor()`, aby dowiedzieć się więcej o tej funkcji.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Jak się okazuje, istnieje tylko słaba korelacja między Miastem a Ceną. Jednakże, korelacja między Opakowaniem a jego Ceną jest nieco lepsza. To ma sens, prawda? Zazwyczaj, im większe pudełko z produktami, tym wyższa cena.

Skoro już o tym mowa, spróbujmy również zwizualizować macierz korelacji wszystkich kolumn za pomocą pakietu `corrplot`.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 O wiele lepiej.

Dobre pytanie, które teraz można zadać na podstawie tych danych, to: '`Jakiej ceny mogę się spodziewać za dany pakiet dyni?`' Przejdźmy od razu do rzeczy!

> Uwaga: Kiedy **`bake()`** wykonujesz na przygotowanym przepisie **`pumpkins_prep`** z **`new_data = NULL`**, uzyskujesz przetworzone (tj. zakodowane) dane treningowe. Jeśli miałbyś inny zbiór danych, na przykład zbiór testowy, i chciałbyś zobaczyć, jak przepis go przetworzy, wystarczyłoby wykonać **`bake()`** na **`pumpkins_prep`** z **`new_data = test_set`**.

## 4. Zbuduj model regresji liniowej

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografika autorstwa Dasani Madipalli</figcaption>


<!--![Infografika autorstwa Dasani Madipalli](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.pl.png){width="800"}-->


Teraz, gdy stworzyliśmy przepis i faktycznie potwierdziliśmy, że dane będą odpowiednio przetwarzane wstępnie, zbudujmy model regresji, aby odpowiedzieć na pytanie: `Jakiej ceny mogę się spodziewać dla danego opakowania dyni?`

#### Wytrenuj model regresji liniowej na zbiorze treningowym

Jak zapewne już zauważyłeś, kolumna *price* jest zmienną `wynikową`, podczas gdy kolumna *package* jest zmienną `predyktorem`.

Aby to zrobić, najpierw podzielimy dane tak, aby 80% trafiło do zbioru treningowego, a 20% do zbioru testowego. Następnie zdefiniujemy przepis, który zakoduje kolumnę predyktora jako zestaw liczb całkowitych, a potem stworzymy specyfikację modelu. Nie będziemy przygotowywać i piec naszego przepisu, ponieważ już wiemy, że odpowiednio przetworzy dane.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Świetna robota! Teraz, gdy mamy przepis i specyfikację modelu, musimy znaleźć sposób na połączenie ich w jeden obiekt, który najpierw wstępnie przetworzy dane (prep+bake za kulisami), dopasuje model do wstępnie przetworzonych danych, a także umożliwi ewentualne działania post-processingowe. Jak to brzmi dla Twojego spokoju ducha!🤩

W Tidymodels ten wygodny obiekt nazywa się [`workflow`](https://workflows.tidymodels.org/) i przechowuje wszystkie komponenty modelowania w jednym miejscu! To jest to, co w *Pythonie* nazywamy *pipelines*.

Więc połączmy wszystko w workflow!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

Na dodatek, przepływ pracy można dopasować/szkolić w bardzo podobny sposób, jak model.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

Z wyników modelu możemy zobaczyć współczynniki wyuczone podczas treningu. Reprezentują one współczynniki linii najlepszego dopasowania, która daje nam najniższy ogólny błąd między rzeczywistą a przewidywaną zmienną.


#### Ocena wydajności modelu za pomocą zestawu testowego

Czas sprawdzić, jak model sobie poradził 📏! Jak to zrobić?

Teraz, gdy wytrenowaliśmy model, możemy go użyć do przewidywania dla test_set za pomocą `parsnip::predict()`. Następnie możemy porównać te przewidywania z rzeczywistymi wartościami etykiet, aby ocenić, jak dobrze (lub nie!) działa model.

Zacznijmy od przewidywania dla zestawu testowego, a następnie połączmy kolumny z zestawem testowym.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Tak, właśnie wytrenowałeś model i użyłeś go do przewidywań! 🔮 Czy jest dobry? Sprawdźmy, jak działa nasz model!

W Tidymodels robimy to za pomocą `yardstick::metrics()`! W przypadku regresji liniowej skupmy się na następujących metrykach:

-   `Root Mean Square Error (RMSE)`: Pierwiastek kwadratowy z [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). Daje to absolutną miarę w tej samej jednostce co etykieta (w tym przypadku cena dyni). Im mniejsza wartość, tym lepszy model (w uproszczeniu, reprezentuje średnią wartość, o którą przewidywania są błędne!).

-   `Coefficient of Determination (często nazywany R-kwadrat lub R2)`: Względna miara, w której wyższa wartość oznacza lepsze dopasowanie modelu. W istocie ta metryka pokazuje, jak dużą część zmienności między przewidywanymi a rzeczywistymi wartościami etykiety model jest w stanie wyjaśnić.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Tam idzie wydajność modelu. Zobaczmy, czy możemy uzyskać lepszy obraz, wizualizując wykres punktowy przedstawiający pakiet i cenę, a następnie używając przewidywań do nałożenia linii najlepszego dopasowania.

Oznacza to, że będziemy musieli przygotować i przetworzyć zestaw testowy, aby zakodować kolumnę pakietu, a następnie połączyć to z przewidywaniami naszego modelu.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Świetnie! Jak widać, model regresji liniowej niezbyt dobrze uogólnia zależność między opakowaniem a jego odpowiadającą ceną.

🎃 Gratulacje, właśnie stworzyłeś model, który może pomóc przewidzieć cenę kilku odmian dyń. Twoja świąteczna dyniowa grządka będzie piękna. Ale prawdopodobnie możesz stworzyć lepszy model!

## 5. Zbuduj model regresji wielomianowej

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografika autorstwa Dasani Madipalli</figcaption>


<!--![Infografika autorstwa Dasani Madipalli](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.pl.png){width="800"}-->


Czasami nasze dane mogą nie mieć liniowej zależności, ale mimo to chcemy przewidzieć wynik. Regresja wielomianowa może pomóc nam w przewidywaniu bardziej złożonych, nieliniowych zależności.

Weźmy na przykład zależność między opakowaniem a ceną w naszym zestawie danych o dyniach. Chociaż czasami istnieje liniowa zależność między zmiennymi - im większa dynia pod względem objętości, tym wyższa cena - czasami tych zależności nie da się przedstawić jako płaszczyznę lub prostą linię.

> ✅ Oto [kilka innych przykładów](https://online.stat.psu.edu/stat501/lesson/9/9.8) danych, które mogą korzystać z regresji wielomianowej
>
> Spójrz jeszcze raz na zależność między odmianą a ceną na wcześniejszym wykresie. Czy ten wykres punktowy wydaje się być koniecznie analizowany za pomocą prostej linii? Być może nie. W takim przypadku możesz spróbować regresji wielomianowej.
>
> ✅ Wielomiany to wyrażenia matematyczne, które mogą składać się z jednej lub więcej zmiennych i współczynników

#### Wytrenuj model regresji wielomianowej na zbiorze treningowym

Regresja wielomianowa tworzy *zakrzywioną linię*, aby lepiej dopasować dane nieliniowe.

Sprawdźmy, czy model wielomianowy będzie lepiej przewidywał wyniki. Postępujemy według podobnej procedury jak wcześniej:

-   Utwórz przepis, który określa kroki wstępnego przetwarzania danych, aby przygotować je do modelowania, np. kodowanie predyktorów i obliczanie wielomianów stopnia *n*

-   Zbuduj specyfikację modelu

-   Połącz przepis i specyfikację modelu w jeden przepływ pracy

-   Utwórz model, dopasowując przepływ pracy

-   Oceń, jak dobrze model działa na danych testowych

Zaczynajmy!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Ocena wydajności modelu

👏👏Stworzyłeś model wielomianowy, teraz czas na przewidywania na zestawie testowym!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Hurra, oceńmy, jak model wypadł na testowym zestawie danych, używając `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Znacznie lepsza wydajność.

`rmse` zmniejszyło się z około 7 do około 3, co wskazuje na mniejszy błąd między rzeczywistą ceną a przewidywaną ceną. Można to *luźno* interpretować jako średni błąd prognoz wynoszący około 3 USD. `rsq` wzrosło z około 0.4 do 0.8.

Wszystkie te wskaźniki pokazują, że model wielomianowy działa znacznie lepiej niż model liniowy. Świetna robota!

Zobaczmy, czy możemy to zwizualizować!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Widać krzywą, która lepiej dopasowuje się do Twoich danych! 🤩

Możesz ją wygładzić, przekazując formułę wielomianową do `geom_smooth`, na przykład tak:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Podobnie jak gładka krzywa!🤩

Oto jak możesz dokonać nowej prognozy:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


Predykcja za pomocą `modelu wielomianowego` ma sens, biorąc pod uwagę wykresy rozrzutu dla `price` i `package`! A jeśli ten model jest lepszy od poprzedniego, patrząc na te same dane, musisz uwzględnić budżet na te droższe dynie!

🏆 Brawo! Stworzyłeś dwa modele regresji w jednej lekcji. W ostatniej części dotyczącej regresji nauczysz się o regresji logistycznej, aby określać kategorie.

## **🚀Wyzwanie**

Przetestuj kilka różnych zmiennych w tym notebooku, aby zobaczyć, jak korelacja wpływa na dokładność modelu.

## [**Quiz po wykładzie**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Przegląd i samodzielna nauka**

W tej lekcji nauczyliśmy się o regresji liniowej. Istnieją inne ważne rodzaje regresji. Przeczytaj o technikach Stepwise, Ridge, Lasso i Elasticnet. Dobrym kursem do nauki w celu zgłębienia tematu jest [Stanford Statistical Learning course](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Jeśli chcesz dowiedzieć się więcej o korzystaniu z niesamowitego frameworka Tidymodels, sprawdź poniższe zasoby:

-   Strona Tidymodels: [Rozpocznij pracę z Tidymodels](https://www.tidymodels.org/start/)

-   Max Kuhn i Julia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **PODZIĘKOWANIA DLA:**

[Allison Horst](https://twitter.com/allison_horst?lang=en) za stworzenie niesamowitych ilustracji, które sprawiają, że R jest bardziej przyjazne i angażujące. Znajdź więcej ilustracji w jej [galerii](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za źródło autorytatywne. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
